In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import string
import jamspell
import matplotlib.pyplot as plt
from dataset_prepare import ToStrLower, DropDuplicates, FilterByQuantile
from autocompleter import AutoCompleterBaseline, AutoCompleteBySuffix, AutoCompleteByInvertedIndex
from inverted_index import InvertedIndex
from tries import make_trie

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
col_list = ['wbuser_id', 'UQ', 'cnt', 'lem_uq']
df = pd.read_csv("data/s_hist_4mil_preprocessed.csv", usecols=col_list)

## Filter data

In [9]:
# filter out duplicates by wbuser_id and UQ
subset_drop = ["wbuser_id", "UQ"]
drop_duplicates_filter = DropDuplicates(subset_drop)
df = drop_duplicates_filter.apply(df)

DropDuplicates: dataset size 3975699 -> 3764740


In [10]:
UPPER_QUANTILE = 0.95
LOWER_QUANTILE = 0.001
quantile_filter = FilterByQuantile('cnt', UPPER_QUANTILE)
df = quantile_filter.apply(df)

Lower quantile: 0. Upper quantile: 30463.0
FilterByQuantile: dataset size 3764740 -> 3576506


In [11]:
popularity = df.groupby("UQ").size().reset_index(name='popularity')

In [6]:
# popularity['popularity'] = popularity['popularity'] / popularity['popularity'].max()

In [12]:
uq_to_lemma_map = {}
for idx, row in df[['UQ', "lem_uq"]].iterrows():
    if row['UQ'] in uq_to_lemma_map:
        continue
    uq_to_lemma_map[row['UQ']] = row['lem_uq']

In [13]:
del df
import gc
gc.collect()

191

In [156]:
# popularity[(popularity['UQ'].str.contains("ноутбук")) & (popularity['UQ'].str.contains("игровой"))]

### Create inverted index

In [15]:
query_popularity_corpus = zip(popularity['UQ'], popularity['popularity'])

In [16]:
inverted_index = InvertedIndex(query_to_lemma_map=uq_to_lemma_map)
index = inverted_index.process_corpus(query_popularity_corpus)

### Create tree

In [17]:
trie = make_trie(popularity['UQ'].values)

Making database for queries... It length: 1935747


### Create spellchecker

In [18]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('data/jamspell_ru_model_subtitles.bin')

True

### Create autocompleter

In [146]:
autocomplete_by_inverted_index = AutoCompleteByInvertedIndex(index, trie, corrector, queries_score_thr=0.1, first_prefix=5, max_candidates=50)

In [147]:
autocomplete_by_inverted_index.query("ноутбук", max_n=10)

Original query score for ноутбук: 0.1440138808559861
Got 100 raw results


[Query indx: 1169880. Words: ноутбук. Score: 402. Lemma: ['ноутбук'],
 Query indx: 1170019. Words: ноутбук hp. Score: 45. Lemma: ['ноутбук', 'hp'],
 Query indx: 1170214. Words: ноутбук игровой. Score: 45. Lemma: ['ноутбук', 'игровой'],
 Query indx: 1169959. Words: ноутбук asus. Score: 28. Lemma: ['ноутбук', 'asus'],
 Query indx: 1170093. Words: ноутбук lenovo. Score: 23. Lemma: ['ноутбук', 'lenovo'],
 Query indx: 1605781. Words: сумка для ноутбука. Score: 191. Lemma: ['сумка', 'для', 'ноутбук'],
 Query indx: 1332879. Words: подставка для ноутбука. Score: 101. Lemma: ['подставка', 'для', 'ноутбук'],
 Query indx: 1595926. Words: столик для ноутбука. Score: 56. Lemma: ['столик', 'для', 'ноутбук'],
 Query indx: 1464887. Words: рюкзак для ноутбука. Score: 52. Lemma: ['рюкзак', 'для', 'ноутбук'],
 Query indx: 1826081. Words: чехол для ноутбука. Score: 49. Lemma: ['чехол', 'для', 'ноутбук']]

In [148]:
autocomplete_by_inverted_index.query("польто", max_n=10)

Original query score for польто: 0.02631578947368418
Fixed word польто -> пальто
Got 100 raw results


[Query indx: 1235887. Words: пальто женское осеннее. Score: 3240. Lemma: ['пальто', 'женский', 'осенний'],
 Query indx: 1235137. Words: пальто женское. Score: 1708. Lemma: ['пальто', 'женский'],
 Query indx: 1233586. Words: пальто. Score: 693. Lemma: ['пальто'],
 Query indx: 1237344. Words: пальто мужское. Score: 446. Lemma: ['пальто', 'мужской'],
 Query indx: 1236312. Words: пальто женское шерстяное. Score: 313. Lemma: ['пальто', 'женский', 'шерстяной'],
 Query indx: 1236561. Words: пальто зимнее женское. Score: 308. Lemma: ['пальто', 'зимний', 'женский'],
 Query indx: 1235410. Words: пальто женское демисезонное. Score: 262. Lemma: ['пальто', 'женский', 'демисезонный'],
 Query indx: 1234768. Words: пальто для девочки. Score: 245. Lemma: ['пальто', 'для', 'девочка'],
 Query indx: 1236476. Words: пальто зимнее. Score: 229. Lemma: ['пальто', 'зимний'],
 Query indx: 1239028. Words: пальто халат. Score: 137. Lemma: ['пальто', 'халат']]

In [149]:
autocomplete_by_inverted_index.query("Игровой ноутбук", max_n=10)

Original query score for игровой: 0.11896243291592123
Original query score for ноутбук: 0.1440138808559861
Got 34 raw results


[Query indx: 705308. Words: игровой ноутбук. Score: 15. Lemma: ['игровой', 'ноутбук'],
 Query indx: 705309. Words: игровой ноутбук 540x. Score: 1. Lemma: ['игровой', 'ноутбук', '540'],
 Query indx: 705310. Words: игровой ноутбук acer. Score: 1. Lemma: ['игровой', 'ноутбук', 'acer'],
 Query indx: 705311. Words: игровой ноутбук hp. Score: 1. Lemma: ['игровой', 'ноутбук', 'hp'],
 Query indx: 705312. Words: игровой ноутбук lenovo. Score: 1. Lemma: ['игровой', 'ноутбук', 'lenovo'],
 Query indx: 705314. Words: игровой ноутбук msi. Score: 1. Lemma: ['игровой', 'ноутбук', 'msi'],
 Query indx: 1170215. Words: ноутбук игровой 1060. Score: 1. Lemma: ['ноутбук', 'игровой', '1060'],
 Query indx: 1170217. Words: ноутбук игровой gtx3050. Score: 1. Lemma: ['ноутбук', 'игровой', 'gtx3050'],
 Query indx: 1077421. Words: мощный игровой ноутбук. Score: 1. Lemma: ['мощный', 'игровой', 'ноутбук'],
 Query indx: 1170220. Words: ноутбук игровой i. Score: 1. Lemma: ['ноутбук', 'игровой', 'i']]

In [155]:
autocomplete_by_inverted_index.query("польто красивое", max_n=10)

Original query score for польто: 0.02631578947368418
Fixed word польто -> пальто
Original query score for красивое: 0.10833333333333328
Got 7 raw results


[Query indx: 1237121. Words: пальто красивое женское. Score: 1. Lemma: ['пальто', 'красивый', 'женский'],
 Query indx: 1237122. Words: пальто красивое необыяное. Score: 1. Lemma: ['пальто', 'красивый', 'необыяной'],
 Query indx: 886826. Words: красивое пальто плащ женское. Score: 1. Lemma: ['красивый', 'пальто', 'плащ', 'женский'],
 Query indx: 886827. Words: красивое пальто стеганое. Score: 1. Lemma: ['красивый', 'пальто', 'стёганый'],
 Query indx: 886806. Words: красивое женское пальто утепленное. Score: 1. Lemma: ['красивый', 'женский', 'пальто', 'утеплить'],
 Query indx: 1236953. Words: пальто капюшон красивое. Score: 1. Lemma: ['пальто', 'капюшон', 'красивый'],
 Query indx: 886813. Words: красивое кашемировое белое пальто. Score: 1. Lemma: ['красивый', 'кашемировый', 'белый', 'пальто']]

In [151]:
word = "ко"
candidates = autocomplete_by_inverted_index.get_candidates(word)
candidates_queries = autocomplete_by_inverted_index.get_queries_from_inverted_index(candidates)
candidate_queries_score = autocomplete_by_inverted_index.calculate_score_for_candidates_queries(
    candidates_queries
)

In [1]:
# sorted(candidates_queries, key=lambda x: x.popularity, reverse=False)